In [1]:
import os

import pandas as pd
import numpy as np

import tqdm 
tqdm.tqdm_notebook().pandas()

# 中文NLP工具
from string import punctuation
import re


In [2]:
df = pd.read_csv("../output/data.csv")

In [5]:
# Prepare Texts
texts = df['text_preprocessed'].values.tolist()

### 百度AI API

In [6]:
# encoding:utf-8
import os
import urllib 
import json
import requests 
import time

def get_AT(AK,SK):
    # client_id 为官网获取的AK， client_secret 为官网获取的SK
    host = "https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={}&client_secret={}".format(AK,SK)

    response = requests.get(host)
    if response:
        print('Successfully get access_token')
        return response.json()['access_token']
    else:
        print('Unable get access_token, retry in 3s...')
        time.sleep(3)
        return get_AT(AK,SK)

def request_api(text, url):
    """
    UTF-8 encode, use given url
    """
    # send request
    data = json.dumps({'text':text}).encode('utf-8')
    request = urllib.request.Request(url=url,data=data)
    request.add_header('Content-Type', 'application/json')
    
    # get answer
    response = urllib.request.urlopen(request)
    if response:
        content = response.read().decode('utf-8')

        rdata = json.loads(content)
        return rdata
    else:
        print('Error, retry in 3s...')
        time.sleep(3)
        return request_api(text, api, AT)


In [7]:
# Config  
AK = " your acces key 去百度AI 申请"
SK = "your security key 同上"

AT = get_AT(AK,SK) # Get Date 2020/05/29  
sentiment_classify_api = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify"

# prepare data and url
url = sentiment_classify_api+"?charset=UTF-8&access_token="+AT

In [8]:
res_list = []
items_dic = {
    'positive_prob':[],
    'negative_prob':[],
    'confidence':[],
    'sentiment':[]
    }

# POST baidu API
for s in tqdm.notebook.tqdm(texts):
    res_list.append(request_api(s, url))

# flatten results
for res in res_list:
    try:
        res_items = res['items'][0]
    except:
        res_items = {'positive_prob': 0, 'confidence': 0, 'negative_prob': 0, 'sentiment': 1}
    
    for key in res_items.keys():
        items_dic[key].append(res_items[key])   

KeyError: 'items'

In [35]:
# update result to df
for key in items_dic.keys():
    df[key] = items_dic[key]

df['error_sentiment'] = 0
df[df['confidence'] == 0]['error_sentiment'] = 1

df['sentiment'].value_counts()

In [37]:
# df.to_csv('../data/sentiment.csv')